In [10]:

import pandas as pd
import networkx as nx
import os
 

GSA without Policies

In [ ]:
from ema_workbench import (
    Samplers
)

In [ ]:
from ema_workbench import MultiprocessingEvaluator #here use instead the data that has already been generated with a sample of random policies


# Detect number of physical/logical cores
import multiprocessing

total_cores = multiprocessing.cpu_count()  # e.g., 16 logical cores for Ryzen 7700X
desired_cores = total_cores // 2  # use 50% of your CPU (adjust as needed)

N = 512  # Sobol sample base size
with MultiprocessingEvaluator(model, n_processes=desired_cores) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(
        scenarios=N,
        policies=[zero_policy],
        uncertainty_sampling=Samplers.SOBOL
    )

In [ ]:
outcomes

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
import numpy as np

# Step 1: Get the SALib problem setup
problem = get_SALib_problem(model.uncertainties)
expected_len = N * (len(problem['names']) + 2)

# Step 2: Initialize result dictionaries
sobol_results_mean = {}
sobol_results_std = {}
sobol_results_final = {}

# Step 3: Loop through outcomes and apply reduction
for outcome_name, values in outcomes.items():
    arr = np.asarray(values)[:expected_len]

    if arr.ndim == 1:
        # Already scalar
        y_mean = arr
        y_std = np.zeros_like(arr)
        y_final = arr
    elif arr.ndim == 2:
        # 2D array like (20480, 3)
        y_mean = arr.mean(axis=1)
        y_std = arr.std(axis=1)
        y_final = arr[:, -1]
    else:
        print(f"⚠️ Skipping unsupported shape: {outcome_name}, shape={arr.shape}")
        continue

    # Run Sobol analysis for each reduction
    sobol_results_mean[outcome_name] = sobol.analyze(problem, y_mean, calc_second_order=False)
    sobol_results_std[outcome_name] = sobol.analyze(problem, y_std, calc_second_order=False)
    sobol_results_final[outcome_name] = sobol.analyze(problem, y_final, calc_second_order=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Extract parameter names from the problem
parameter_names = problem['names']

# Convert a Sobol result dictionary into a tidy DataFrame
def sobol_to_dataframe(sobol_dict, parameter_names, label):
    records = []
    for outcome_name, Si in sobol_dict.items():
        for i, param in enumerate(parameter_names):
            records.append({
                'Outcome': outcome_name,
                'Parameter': param,
                'S1': Si['S1'][i],
                'ST': Si['ST'][i],
                'S1_conf': Si['S1_conf'][i],
                'ST_conf': Si['ST_conf'][i],
                'Reduction': label
            })
    return pd.DataFrame(records)

# Convert all three dictionaries to DataFrames
df_mean = sobol_to_dataframe(sobol_results_mean, parameter_names, 'Mean')
df_std = sobol_to_dataframe(sobol_results_std, parameter_names, 'Std')
df_final = sobol_to_dataframe(sobol_results_final, parameter_names, 'Final')

# Combine them all into one long-form DataFrame
df_all = pd.concat([df_mean, df_std, df_final], ignore_index=True)

# Plotting function for heatmaps
def plot_sobol_heatmap(df, reduction_type):
    pivot = df[df['Reduction'] == reduction_type].dropna(subset=["ST"])\
        .pivot(index='Outcome', columns='Parameter', values='ST')

    if pivot.empty:
        print(f"⚠️ No valid data to plot for reduction type: {reduction_type}")
        return

    plt.figure(figsize=(14, max(6, len(pivot) * 0.4)))
    sns.heatmap(pivot, annot=True, fmt=".2f", cmap='YlGnBu', cbar_kws={'label': 'Total Sobol Index (ST)'})
    plt.title(f"Sobol ST Heatmap ({reduction_type})")
    plt.ylabel("Outcome")
    plt.xlabel("Input Parameter")
    plt.tight_layout()
    plt.show()

# Plot all three heatmaps
plot_sobol_heatmap(df_all, 'Mean')
plot_sobol_heatmap(df_all, 'Std')
plot_sobol_heatmap(df_all, 'Final')